In [43]:
import pandas as pd
import numpy as np 
import statsmodels.api as sm 
from patsy import dmatrices, dmatrix
%matplotlib inline

In [4]:
df = pd.read_excel("TrainExer44.xls")

*** No CODEPAGE record, no encoding_override: will use 'ascii'


first stage

(a) GC = β1 + β2PG + β3RI + ε

(b) Use 2SLS to estimate the price elasticity (β2). Use a constant, RI, RPT, RPN, and RPU as instruments.

In [6]:
df.head(3)

,OBS,GC,PG,RI,RPN,RPT,RPU
0,1970,6.714227,-0.100947,2.943489,0.311872,-0.097374,-0.218002
1,1971,6.765328,-0.137221,2.985432,0.309661,-0.068993,-0.204794
2,1972,6.830206,-0.156660,3.033986,0.268967,-0.061672,-0.233363


In [35]:
y,X = dmatrices("GC ~  PG + RI", df)
original_mod = sm.OLS(y, X).fit()
print (original_mod.summary())

                            OLS Regression Results                            
Dep. Variable:                     GC   R-squared:                       0.987
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     1037.
Date:                Sun, 13 May 2018   Prob (F-statistic):           2.94e-26
Time:                        21:45:08   Log-Likelihood:                 71.096
No. Observations:                  30   AIC:                            -136.2
Df Residuals:                      27   BIC:                            -132.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.9860      0.081     61.479      0.0

In [7]:
y1, X1 = dmatrices("PG ~ RI + RPT + RPN + RPU", df)
stage_1_mod = sm.OLS(y1, X1).fit()
print (stage_1_mod.summary())

                            OLS Regression Results                            
Dep. Variable:                     PG   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.869
Method:                 Least Squares   F-statistic:                     48.97
Date:                Sun, 13 May 2018   Prob (F-statistic):           1.80e-11
Time:                        19:43:18   Log-Likelihood:                 38.812
No. Observations:                  30   AIC:                            -67.62
Df Residuals:                      25   BIC:                            -60.62
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.7410      0.834      9.285      0.0

In [8]:
PG_explained = stage_1_mod.predict(X1)

In [19]:
X2 = sm.add_constant(np.vstack([PG_explained, df.RI.values]).T)
y2 = df.GC.values

In [21]:
stage_2_mod = sm.OLS(y2, X2).fit()
print (stage_2_mod.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.964
Method:                 Least Squares   F-statistic:                     393.3
Date:                Sun, 13 May 2018   Prob (F-statistic):           1.08e-20
Time:                        19:48:02   Log-Likelihood:                 56.859
No. Observations:                  30   AIC:                            -107.7
Df Residuals:                      27   BIC:                            -103.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.0137      0.134     37.456      0.0

In [27]:
resid_s2 = y2 - stage_2_mod.predict(sm.add_constant(df[["PG", "RI"]]))

In [36]:
resid_s2_test = y2 - stage_2_mod.predict(X)

In [28]:
y3 = resid_s2
X3 = dmatrix("RI + RPT + RPN + RPU +RI", df)

In [46]:
sargon_mod = sm.OLS(y3, X3).fit()
print (sargon_mod.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                 -0.039
Method:                 Least Squares   F-statistic:                    0.7267
Date:                Sun, 13 May 2018   Prob (F-statistic):              0.582
Time:                        21:47:09   Log-Likelihood:                 72.519
No. Observations:                  30   AIC:                            -135.0
Df Residuals:                      25   BIC:                            -128.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2098      0.271     -0.774      0.4

$SargonStats = 30 * 0.104 = 3.12$